**Завдання:**

Створіть модель згорткової нейронної мережі із трома шарами. Підключіть Tensorboard, та відобразіть в ньому зміну розподілу ваг моделі, обчислювальний граф моделі та зміну її метрик.

In [12]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
import datetime

In [14]:
%load_ext tensorboard

In [29]:
# Определяем путь к папке с датасетом фруктов
train_dir = 'C:/Users/админ/PycharmProjects/PDS2/PDS2_Kofanov/My_project/Old_datasets/dataset_train'
test_dir = 'C:/Users/админ/PycharmProjects/PDS2/PDS2_Kofanov/My_project/Old_datasets/dataset_test'
path_to_save = 'C:/Users/админ/PycharmProjects/PDS2/PDS2_Kofanov/Lesson_68/'

In [16]:
# Определяем параметры для обработки изображений
img_width, img_height = 227, 227
batch_size = 32

In [17]:
# Создаем объекты для генерации изображений
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   shear_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
# Создаем генераторы изображений для обучения и тестирования модели
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

Found 80 images belonging to 20 classes.
Found 20 images belonging to 20 classes.


In [19]:
# Определяем архитектуру модели
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(os.listdir(train_dir)), activation='softmax'))

In [20]:
# Компилируем модель
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_checkpoint_callback = ModelCheckpoint(
    filepath='C:/Users/админ/PycharmProjects/PDS2/PDS2_Kofanov/My_project/Results/{epoch:02d}.h5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

In [21]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S")) 
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq =  1, write_graph = False,write_images = False)

In [25]:
# Обучаем модель
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          validation_data=test_generator,
          validation_steps=test_generator.samples // batch_size,
          epochs=100)

Epoch 1/100
2/2 [==============================] - 5s 2s/step - loss: 2.7641 - accuracy: 0.0938
Epoch 2/100
2/2 [==============================] - 4s 1s/step - loss: 2.7230 - accuracy: 0.0833
Epoch 3/100
2/2 [==============================] - 5s 2s/step - loss: 2.7817 - accuracy: 0.1250
Epoch 4/100
2/2 [==============================] - 5s 2s/step - loss: 2.5251 - accuracy: 0.2500
Epoch 5/100
2/2 [==============================] - 4s 2s/step - loss: 2.7026 - accuracy: 0.2292
Epoch 6/100
2/2 [==============================] - 4s 2s/step - loss: 2.4453 - accuracy: 0.2083
Epoch 7/100
2/2 [==============================] - 5s 2s/step - loss: 2.6790 - accuracy: 0.1094
Epoch 8/100
2/2 [==============================] - 5s 2s/step - loss: 2.6788 - accuracy: 0.1406
Epoch 9/100
2/2 [==============================] - 5s 2s/step - loss: 2.5609 - accuracy: 0.1875
Epoch 10/100
2/2 [==============================] - 4s 1s/step - loss: 2.4958 - accuracy: 0.1458
Epoch 11/100
2/2 [=====================

In [26]:
tensorboard --logdir 'C:/Users/админ/PycharmProjects/PDS2/PDS2_Kofanov/Lesson_68/logs/' 

In [27]:
# Оцениваем точность модели на тестовых данных
accuracy = model.evaluate(test_generator)[1]
print('Accuracy:', accuracy)

1/1 [==============================] - 1s 615ms/step - loss: 2.1354 - accuracy: 0.3500
Accuracy: 0.3499999940395355


In [30]:
model.save(path_to_save + '/fruit_classifier_color_100.h5')